# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
             cada variable es una columna y cada observación (o caso) es una fila.   
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [2]:
import pandas as pd
import numpy as np
import gzip
import openpyxl
import pickle

from io import StringIO
import requests


Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [5]:
# URL del archivo CSV comprimido en formato Gzip
url = 'https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true'

# Descargar el archivo comprimido en formato Gzip
response = requests.get(url)
data = gzip.decompress(response.content).decode('utf-8')

# Crear un objeto StringIO para leer el CSV con pandas
loans = pd.read_csv(StringIO(data))

loans


/tmp/ipykernel_285/2403947836.py:9: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv(StringIO(data))


,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2,37842129,NaN,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,3,37612354,NaN,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,4,37662224,NaN,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,22454240,NaN,8400.0,8400.0,8400.0,36 months,9.17,267.79,B,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99996,99996,11396920,NaN,10000.0,10000.0,10000.0,36 months,12.99,336.90,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99997,99997,8556176,NaN,30000.0,30000.0,30000.0,60 months,20.99,811.44,E,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99998,99998,24023408,NaN,8475.0,8475.0,8475.0,36 months,24.99,336.92,F,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [6]:
column_types=loans.dtypes
#print (column_types)

# Crear un DataFrame más legible
column_info = pd.DataFrame({'Columna': column_types.index, 'Tipo': column_types.values})

# Imprimir el DataFrame con la información
print(column_info)

                   Columna     Tipo
0               Unnamed: 0    int64
1                       id    int64
2                member_id  float64
3                loan_amnt  float64
4              funded_amnt  float64
..                     ...      ...
146      settlement_status   object
147        settlement_date   object
148      settlement_amount  float64
149  settlement_percentage  float64
150        settlement_term  float64

[151 rows x 2 columns]


## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [21]:
datos_dict = pd.read_excel('https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']
pd.set_option('display.max_rows', None)

In [7]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [13]:
# COdigo guardar

# Guardar DataFrame en formato pickle
with open('datos.pkl', 'wb') as file:
    pickle.dump(loans, file)

In [14]:
# Codigo para cargar

# Cargar DataFrame desde el archivo pickle
with open('datos.pkl', 'rb') as file:
    loaded_df = pickle.load(file)
    
    
print("DataFrame cargado desde pickle:")
print(loaded_df)

DataFrame cargado desde pickle:
       Unnamed: 0        id  member_id  loan_amnt  funded_amnt  \
0               0  38098114        NaN    15000.0      15000.0   
1               1  36805548        NaN    10400.0      10400.0   
2               2  37842129        NaN    21425.0      21425.0   
3               3  37612354        NaN    12800.0      12800.0   
4               4  37662224        NaN     7650.0       7650.0   
...           ...       ...        ...        ...          ...   
99995       99995  22454240        NaN     8400.0       8400.0   
99996       99996  11396920        NaN    10000.0      10000.0   
99997       99997   8556176        NaN    30000.0      30000.0   
99998       99998  24023408        NaN     8475.0       8475.0   
99999       99999  24023398        NaN    25000.0      25000.0   

       funded_amnt_inv        term  int_rate  installment grade  ...  \
0              15000.0   60 months     12.39       336.64     C  ...   
1              10400.0   36 mon

## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

Columnas eliminadas vacías

In [8]:
# Manejos de tipos 1
loans = loans.drop('member_id', axis=1)
loans = loans.drop('desc', axis=1)
loans = loans.drop('annual_inc_joint', axis=1)
loans = loans.drop('dti_joint', axis=1)
loans = loans.drop('verification_status_joint', axis=1)

encabezados = loans.columns.tolist()
print(encabezados)

col=['open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc']
loans = loans.drop(columns=col, errors='ignore')

col2=['desc','inq_fi', 'total_cu_tl', 'inq_last_12m']
loans = loans.drop(columns=col2, errors='ignore')

col3=['revol_bal_joint','sec_app_fico_range_low','sec_app_fico_range_high','sec_app_earliest_cr_line','sec_app_inq_last_6mths','sec_app_mort_acc','sec_app_open_acc','sec_app_revol_util','sec_app_open_act_il','sec_app_num_rev_accts','sec_app_chargeoff_within_12_mths','sec_app_collections_12_mths_ex_med','sec_app_mths_since_last_major_derog']
loans = loans.drop(columns=col3, errors='ignore')

#pymnt_plan tiene el mismo valor n para todos los datos, por lo tanto, no brinda información particular y para facilidad, se euitará
loans = loans.drop(columns='pymnt_plan', errors='ignore')

# Tu codigo aqui

['Unnamed: 0', 'id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',

In [17]:
#Checo las colulmas sobrantes
#Tipo de las columnas 
column_types=loans.dtypes
column_info = pd.DataFrame({'Columna': column_types.index, 'Tipo': column_types.values})
# Imprimir el DataFrame con la información
pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
print(column_info)


                                        Columna     Tipo
0                                    Unnamed: 0    int64
1                                            id    int64
2                                     loan_amnt  float64
3                                   funded_amnt  float64
4                               funded_amnt_inv  float64
5                                          term   object
6                                      int_rate  float64
7                                   installment  float64
8                                         grade   object
9                                     sub_grade   object
10                                    emp_title   object
11                                   emp_length   object
12                               home_ownership   object
13                                   annual_inc  float64
14                          verification_status   object
15                                      issue_d   object
16                             

In [20]:
#Convierto subgrade a string, todos los tipos está compuestos por una variable y un número
loans['sub_grade'] = loans['sub_grade'].astype(str)
#Covierto grade a string, pues solo es una consonante igualmente addr_state
loans['grade'] = loans['sub_grade'].astype(str)
loans['addr_state'] = loans['addr_state'].astype(str)
#Las columnas de las cantidades son todas enteros, por lo tanto asumo que no hay préstamos con decimales           
loans['loan_amnt'] = loans['loan_amnt'].astype(int)
loans['funded_amnt'] = loans['funded_amnt'].astype(int)
#Convierto las fechas a datetime
loans['issue_d'] = pd.to_datetime(loans['issue_d'], format='%b-%Y')
loans['earliest_cr_line'] = pd.to_datetime(loans['earliest_cr_line'], format='%b-%Y')
loans['last_pymnt_d'] = pd.to_datetime(loans['last_pymnt_d'], format='%b-%Y')
loans['next_pymnt_d'] = pd.to_datetime(loans['next_pymnt_d'], format='%b-%Y')
loans['last_credit_pull_d'] = pd.to_datetime(loans['last_credit_pull_d'], format='%b-%Y')
loans['last_pymnt_d'] = pd.to_datetime(loans['last_pymnt_d'], format='%b-%Y')


#Los meses del ingreso se cuentan como meses enteros
loans['inq_last_6mths'] = loans['inq_last_6mths'].astype(int)
loans['mths_since_last_delinq'] = loans['mths_since_last_delinq'].astype(int)
loans['mths_since_last_record'] = loans['mths_since_last_record'].astype(int)
loans['open_acc'] = loans['open_acc'].astype(int)
loans['total_acc'] = loans['total_acc'].astype(int)


loans['initial_list_status'] = loans['initial_list_status'].astype(str)

#Casteo a Booleanos
loans['verification_status'] == 'verified'
     



In [ ]:
#Manejo de Missings Todas son enteero
# Reemplazar valores nulos con numpy.nan
loans['inq_last_6mths'] = loans['inq_last_6mths'].replace('', np.nan)
loans['mths_since_last_delinq'] = loans['mths_since_last_delinq'].replace('', np.nan)
loans['mths_since_last_record'] = loans['mths_since_last_record'].replace('', np.nan)
loans['mths_since_last_major_derog'] = loans['mths_since_last_major_derog'].replace('', np.nan)


#Relleno las columnas de las fechas
loans['next_pymnt_d'] = loans['mi_columna'].fillna(np.nan)


## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [ ]:
#Json

estrategias_json={"id","member_id","loan_amnt", "delinq2Yrs":{"estartegia":"Elegir a las personas que se hayan retrasado en el pago", "valor": ">=1"},
 "acc_now_delinq":{"estrategia":"Personas con más en una cuenta que deba", "valor":">=1"}, 
 "revol_util":{"estrategia": "Encotrar los créditos mayores al promedio", "valor":">=mean"},
 "empLength":{"estrategia":"Estabilidad de la personas por sus años de empleo", "valor":"<=1"},
  "loan_amnt":{"estrategia": "Préstamo mayor al promedio", "valor":">=mean"},
 "pub_rec_bankruptcies":{"estrategia":" Rgistros públicos desfavorables", "valor":">=1"},
"funded_amnt", "funded_amnt_inv","term	int_rate", "installment", "grade","sub_grade","emp_title", "emp_length"
"home_ownership","annual_inc", "verification_status","issue_d", "loan_status", "pymnt_plan","desc","purpose",
"title", "zip_code", "addr_state","earliest_cr_line","fico_range_low", "fico_range_high","inq_last_6mths",
"mths_since_last_delinq","mths_since_last_record", "open_acc", "pub_rec", "revol_bal","revol_util","total_acc",
"initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt","total_pymnt_inv",	"total_rec_prncp",	"total_rec_int"}
# Guardar el JSON en un archivo
with open('estrategias_imputacion.json', 'w') as json_file:
    json.dump(estrategias_json, json_file)

print("JSON guardado exitosamente.")

# Cargar el JSON desde el archivo
with open('estrategias_imputacion.json', 'r') as json_file:
    estrategias_cargadas = json.load(json_file)

# Mostrar el JSON cargado
print("JSON cargado:")
print(estrategias_cargadas)



### Codigo para salvar y cargar JSONs